# type

In [1]:
num = 10

In [2]:
num.__class__

int

In [3]:
num.__class__.__class__

type

In [13]:
type("MyClass", (), {})

__main__.MyClass

# Metaclass

In [14]:
class BaseModel(type):
    def __new__(cls, name: str, bases: tuple, attr: dict):
        attr['info'] = 'This is a metaclass'
        return super().__new__(BaseModel, name, bases, attr)

class Test(metaclass=BaseModel):
    def __init__(self) -> None:
        pass

test = Test()

In [15]:
test.info # 此时虽然没有对类Test进行初始化赋值，但是已经有了info这个属性

'This is a metaclass'

In [12]:
from typing import Any


class ListMeta(type):
    def __new__(cls, name: str, bases: tuple, attr: dict):
        attr['add'] = lambda self, value: self.append(value)
        return super().__new__(cls, name, bases, attr) # 这里调用的实际上是type()
    def __call__(cls) -> Any:
        print('我被实例化啦')
        return super().__call__()

class MyList(list, metaclass=ListMeta):
    def __init__(self):
        pass

l = MyList()
l.add(1)
l.add(2)


我被实例化啦


In [5]:
from typing import Self

def custom_change(self, val: int):
    self[list(self.keys())[0]] = val
    return self[list(self.keys())[0]]

class DictMeta(type):
    def __new__(cls, name: str, base: tuple, attr: dict) -> Self:
        attr['change'] = custom_change
        return super().__new__(cls, name, base, attr)

class MyDict(dict, metaclass=DictMeta):
    pass
    # MyDict的相关属性在代码执行到MyDict定义的时候就传入元类DictMeta中，而不是在实例化Mydict时
    # name: MyDict
    # base: dict
    # attr: MyDict现有的自带属性
    # def __init__(self)->None:
    #     print('自定义字典类已初始化完毕')

my_dict = MyDict()
my_dict['first'] = 1
print(my_dict)
my_dict.change(3)
print(my_dict)

{'first': 1}
{'first': 3}


In [23]:
# 实现ORM（对象关系映射）
class ModelMetaclass(type):
    def __new__(cls, name, bases, attrs):
        if name == 'BaseModel': # 抽象的基础类不需要映射
            return type.__new__(cls, name, bases, attrs)
        # 注意，这里简单起见，假设所有字段均为字符串，实际需根据字段类型进行定义。
        # attrs.items() 包括所有类属性，我们只关心字段
        fields = {k: v for k, v in attrs.items() if not k.startswith('__') and not callable(v)}
        for k in fields:
            attrs.pop(k) # 移除类属性，因为已经映射到 fields 字典中
        attrs['__fields__'] = fields # 把所有字段保存在 __fields__ 属性中
        return type.__new__(cls, name, bases, attrs)

class BaseModel(metaclass=ModelMetaclass): # 基础类，所有 ORM 映射类应继承此类
    def __init__(self, **kwargs): # 利用 **kwargs 来初始化对象
        for name, value in kwargs.items():
            setattr(self, name, value)

class User(BaseModel): # 创建映射类
    name = 'unknown' # 字段默认值
    email = 'unknown'


user_tb = User(name='Jack', email='jack123@gmail.com', tele=13516269283)
user_tb